In [1]:
# Import Modules
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, TimestampType, DoubleType
from pyspark.sql.functions import coalesce, lit, col, lead, lag
from pyspark.sql.functions import stddev, mean, col
from pyspark.sql.window import Window

from operator import add
from functools import reduce

from googletrans import Translator

# Standard Python Modules
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import re

In [2]:
class DataframeTools:
  """Parent class for manipulating spark dataframes. """
  
  def __init__(self, df):
    self.df = df
    
  
  def update_df(self, new_df):
    """Replace class dataframe with a new one.
    
    Args:
      new_df (dataframe): New dataframe
      
    """
    
    self.df = new_df
  
  
  def append_data(self, new_df):
    """Append another dataframe below the current one. 
    New dataframe must have the same columns as the original dataframe.
    
    Args:
      new_df (dataframe): New dataframe
      
    """
    
    assert self.df.schema == new_df.schema, "Column headers must match!"
    
    print("Current samples: ", self.df.count())
    print("Appending samples: ", new_df.count())
    
    self.df = self.df.union(new_df)
    print("Joined samples: ", self.df.count())
    print("")
  
  
  def is_null(self, dfs):
    """Check all columns in dataframe for null values.
    
    Args:
      dfs (dataframe): Dataframe to be checked on
      
    """
    
    print("Number of samples with a null value across columns:")
    for col in dfs.schema:
      head = col.name
      #print(dfs[head].isNull() == True)
      print(head, dfs.where(dfs[head].isNull() == True).count())
    print("")
    
    
  def null2zero(self, head, dfs):
    """Change null values to zero in column
    
    Args:
      head (str): Column name
      dfs (dataframe): Dataframe given
    
    Returns:
      dataframe with replaced null values
      
    """
    
    print("Replacing null values with zero...\n")
    dfs = dfs.na.fill(0, (head))
    return dfs

  
  def translate_col(self, head, dfs, src='no', dest='en'):
    """Translate a string column of a dataframe from src language to dest language
    
    Args:
      head (str): Column name
      dfs (dataframe): Dataframe given
      src (str): Source language to translate from
      dest (str): Target language to translate to
    
    Returns:
      new_dfs (dataframe): dataframe with translated column
      translation_dict (dict): dictionary of distinct strings translated
      
    """
    
    # Translate descriptions
    translator = Translator()
    # Select distinct comments
    n = dfs.select(dfs[head]).distinct().rdd.map(lambda x: x[head]).collect()
    # Make a dictionary to translate distinct comments
    translation_dict = {col : translator.translate(col, src=src, dest=dest).text for col in n}
    # Utilise spark method and replace all norweigan comments with translated ones
    new_dfs = dfs.na.replace(translation_dict, 1, head)
    return new_dfs, translation_dict
  
  
  def ts_overlay_records(self, df_ts, df_records, head, filt_dict=None, translate_dict=None):
    """Given a ts dataframe, make a new column and match corresponding records according to time.
       Pass in filt_dict and translate_dict to be able to group up similar records.
       
    Args:
      df_ts (dataframe): Time series dataframe
      df_records (dataframe): Time series dataframe whose values contain records (e.g. string descriptions)
      head (str): Column name of the records in df_records
      filt_dict (dict): Dictionary of similar substrings found in similar records and it's new collective record
      translate_dict (dict): Dictionary of translated distinct records
      
    Returns:
      new_df (dataframe): dataframe containing new column of time correlated records to the time series and other useful columns
    
    """

    # Make a new column where datetime precision: daily. (because interferences are recorded daily)
    df = df_ts.select(
      df_ts["datetime"].alias("datetime_orig"),
      (F.round(F.unix_timestamp(F.col("datetime")) / 86400) * 86400).cast("timestamp").alias("datetime"),
      df_ts["value"]
    )

    new_df = df.join(df_records, on=['datetime'], how='left_outer')
    
    if (filt_dict is not None) and (translate_dict is not None):
      print("\nCollecting similar comments given the filt_dict...")
      group_dict = {}

      for comment in translate_dict.values():
        for abrv, group_comment in filt_dict.items():
          if abrv in comment:
            group_dict[comment] = group_comment
            #print(comment, " : ", group_comment)

      new_df = new_df.withColumn("Grouped", new_df[head])
      new_df = new_df.na.replace(group_dict, 1, "Grouped")
      new_df = new_df.na.fill("No Records", "Grouped")
    
    new_df = new_df.na.fill("No Records", head)
    
    new_df = new_df.drop("datetime")
    new_df = new_df.withColumnRenamed("datetime_orig", "datetime")
    
    return new_df
    
    
  def discretise_col(self, dfs, head):
    """Make new column for a dataframe which numerically discretises a descriptive column.

    Args: 
      dfs (dataframe): Dataframe which contains: datetime, value, Description, Grouped
      head (str): Name of the column to discretise

    Returns:
      dataframe with column of discretised values of the descriptive column

    """

    # Make column by numerically discretising distinct comments
    grouped_desc_list = dfs.select(dfs[head]).distinct().rdd.map(lambda x: x[head]).collect()

    # Convert comments to discrete values
    numerate = {str(val) : str(i) for i, val in enumerate(grouped_desc_list)}

    new_dfs = dfs.withColumn("Discrete_str", dfs[head])

    new_dfs = new_dfs.na.replace(numerate, 1, "Discrete_str")
    new_dfs = new_dfs.withColumn("Discrete", new_dfs["Discrete_str"].cast(IntegerType())).drop("Discrete_str")
    new_dfs = new_dfs.na.fill(0, ("Discrete"))

    return new_dfs
      
      
  def add_year_col(self, dfs):
    """Add column of year of date to dataframe.

    Args:
      dfs (dataframe): dataframe with datetime

    Returns:
      dataframe with column for year.

    """

    return dfs.withColumn("year", F.year(F.col("datetime")))
  
  
  def merge_duplicate(self, dfs):
    """Collect up duplicated datetimes with different descriptions, and merge the descriptions together.

      Args:
        dfs (dataframe): dataframe in the format: datetime|value|description|groupedDescription

      Returns:
        dataframe with merged descriptions

    """

    reduced = dfs\
        .rdd\
        .map(lambda row: (row[0], [(row[1], row[2], row[3])]))\
        .reduceByKey(lambda x,y: x+y)\
        .map(lambda row: (
                row[0],                                #key i.e. datetime
                row[1][0][0],    #sum(row[1][0]) / len(row[1][0]),       #value, take the average of the values 
                ','.join([str(e[1]) for e in row[1]]), #join up the descriptions
                ','.join([str(e[2]) for e in row[1]])  #join up the grouped descriptions
            )
        )

    schema_red = dfs.schema
    new_dfs = sqlContext.createDataFrame(reduced, schema_red).orderBy("datetime")

    if new_dfs.count() > new_dfs.dropDuplicates(["datetime"]).count():
      raise ValueError('Data has duplicates')

    return new_dfs

  
  def avg_over_period(self, dfs, period="day"):
    """Given a dataframe with datetime column, average over days, weeks, months or years and return new dataframe. 
      
      Args:
        dfs (dataframe): dataframe in the format of: datetime|value|...|...
      
      Returns:
        dataframe in format of: datetime|value  , where value is the averaged value over the period
    
    """
    
    dfs_new = dfs.withColumn(period, F.date_trunc(period, dfs.datetime))
    
    dfs_new = dfs_new\
                  .groupBy(period)\
                  .agg(F.avg("value"))\
                  .orderBy("day")
    
    dfs_new = dfs_new.withColumnRenamed("avg(value)", "value")
    
    return dfs_new
  
  
  def threshold(self, dfs):
    mean, std = dfs.select(F.mean("value"), F.stddev("value")).first()
    
    dfs.where(dfs.value >= )
  
  
  ge2016norm[i] = df.withColumn("value_norm", (col("value") - mean) / std)
  ge2016norm[i] = ge2016norm[i].select(ge2016norm[i]["datetime"], ge2016norm[i]["value_norm"].alias("value"))


In [3]:
class GroupDataTools(DataframeTools):
  """Subclass of Dataframe tools to reorganise dataframes into dictionaries.
    Tools for visualisation and preprocessing included.
  """
  
  def __init__(self, df, df_dict={}):
    super().__init__(df)
    self.df_dict = df_dict
    self.headers = [h.name for h in df.schema]
    
    
  def groupdata(self, dict_group_head, x_head, y_head):
    """Collect rows which contain the same value in dict_group_head column and 
    put them into a dictionary. 
    
    Keys: distinct value, Value: dataframe whose rows contain distinct value
    
    Args:
      dict_group_head (str): column to split the dataframe
      x_head (str): first column (normally time) to reconstruct corresponding dataframe
      y_head (str): second column (normally value) to reconstruct corresponding dataframe
    
    """
    
    # Split dataframe up by given header
    assert dict_group_head in self.headers, "Header does not exist in dataframe!"
    
    unq_items = self.df.select(dict_group_head).distinct()
    n = unq_items.rdd.map(lambda x: x[dict_group_head]).collect()

    for key in n:
      self.df_dict[key] = self.df.orderBy(x_head).where(self.df[dict_group_head] == key).select(self.df[x_head], self.df[y_head])
  
  
  def splitdata_dict(self, regex):
    """Make a dictionary from df_dict given a conditional substring which matches within the keys of df_dict.
    E.g. Return a dictionary of a certain oilwell given from a code in the tag name.

    Args:
      regex (str): substring to match with keys, written in wildcard format for regular expressions
      
    """
    
    out_dict = {}
    
    for (key, val) in self.df_dict.items():
      for reg in regex:
        if re.match(reg, key):
          out_dict[key] = val
      
    return out_dict
    
    
  def decode_keys(self, in_dict, decode_dict):
    """Replace the old keys with new key definitions.

      Args:
        in_dict (dict): input oilwell dictionary containing dataframes
        decode_dict (dict): dictionary of old keys with new definitions

      Returns: None
      
    """
    
    new_dict = {}
    
    for key, val in in_dict.items():
      for k, new_key in decode_dict.items():
        regex = re.compile(k)
        if (re.match(regex, key)):
          #print("Replacing ", key, ", with ", new_key)
          new_dict[new_key] = val
          #for i, j in new_dict.items():
            #print(i, " : ", j)
    print("")
    
    return new_dict
  
  
  def plot_ts(self, title, x_head, y_head, ts_df_list, label_list=["value"]):
    """Plot multiple timeseries dataframe onto a figure, x axis = time, y axis = value.

    Args:
      title (str): Name of dataframe
      x_head (str): Name of column to be plotted along x axis
      y_head (str): Name of column to be plotted along y axis
      ts_df_list (list): list of timeseries dataframes to plot
      label_list (list): list of plot labels

    """
    
    fig, ax = plt.subplots(1, 1, figsize=(24, 10))

    for ts_df, lab in zip(ts_df_list, label_list):
      ts_pd = ts_df.orderBy(x_head).toPandas()

      y = ts_pd[y_head].tolist()
      x = ts_pd[x_head].tolist()

      ax.plot(x, y, ".--", label=lab)

    ax.set_title(title, fontsize=16)
    ax.set_xlabel(x_head, fontsize=16)
    ax.set_ylabel(y_head, fontsize=16)

    ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d %H:%M"))
    ax.xaxis.set_minor_formatter(mdates.DateFormatter("%Y-%m-%d"))

    ax.legend(loc="best")

    fig.tight_layout()
    display(fig)
    

  def weighted_average(self, ts_df, x_head, y_head, offsets, weights):
    """Produce rolling average results of the given ts data with the given specs.

      Args:
        ts_df (dataframe): timeseries dataframe
        x_head (str): header name of x axis in timeseries (e.g. datetime)
        y_head (str): header name of y axis in timeseries (e.g. value)
        offsets (list): list of adjacent values to consider
        weights (list): list of weights applied to offsets

    """
    window = Window.orderBy(x_head)
    v = col(y_head)

    assert len(weights) == len(offsets)

    def value(i):
        if i < 0: return lag(v, -i).over(window)
        if i > 0: return lead(v, i).over(window)
        return v

    values = [coalesce(value(i) * w, lit(0))/len(offsets) for i, w in zip(offsets, weights)]

    return reduce(add, values, lit(0))
  
  
  def view_moving_avg(self, title, x_head, y_head, y_label, ts_df, offsets, weights):
    """Wrapper function to view the moving average of the given ts data

      Args:
        title (str): Title of graph
        x_head (str): Header name for x column data
        y_head (str): Header name for y column data
        y_label (str): y axis label
        ts_df (dataframe): Time series dataframe
        offsets (list): list of adjacent values to consider
        weights (list): list of weights applied to offsets

    """

    avg = ts_df.withColumn("avg", weighted_average(ts_df, offsets, weights)).drop(y_head)
    avg = avg.select(avg[x_head],
                     avg["avg"].alias(y_head))

    plot_ts(title, y_label, [avg])
    
    

In [4]:
# Load data into the notebook
df_01 = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/newdump_01.csv')

# Rename and cast types for each column
df_01 = df_01.select(
      df_01["Unnamed: 0"].alias("index"),
      F.to_timestamp(F.col("ts").cast("string"), "dd-MMM-yy HH:mm:ss").alias("datetime"),
      df_01["name"].alias("tag"),
      df_01["value"]
)

df_02 = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/newdump_02.csv')

df_02 = df_02.select(
      df_02["Unnamed: 0"].alias("index"),
      F.to_timestamp(F.col("ts").cast("string"), "dd-MMM-yy HH:mm:ss").alias("datetime"),
      df_02["name"].alias("tag"),
      df_02["value"]
)

#display(df_02.select("tag").distinct())

In [5]:
explore = GroupDataTools(df_01)
explore.append_data(df_02)
explore.is_null(explore.df)
explore.df = explore.null2zero("value", explore.df)
explore.is_null(explore.df)

In [6]:
explore.groupdata("tag", "datetime", "value")

r1 = re.compile('BRA-....-..-07.')
r2 = re.compile('BRA-QT  -15-0077-RAW')
OW1 = explore.splitdata_dict([r1, r2])

r1 = re.compile('BRA-....-..-01.')
r2 = re.compile('BRA-QT  -15-0017-RAW')
OW3 = explore.splitdata_dict([r1, r2])

r1 = re.compile('BRA-....-..-04.')
OW2 = explore.splitdata_dict([r1])

# Make a tag dictionary: Decode the tags!
tag_names = {
              "BRA-PZT........" : "WHP",
              "BRA-TT..-15...." : "WHT",
              "BRA-FI........." : "GLR",
              "BRA-PT..-16...." : "GLP",
              "BRA-PT..-13...." : "DHP",
              "BRA-TT..-13...." : "DHT",
              "BRA-HV........." : "Choke",
              "BRA-QT........." : "ASD"
}

OW1 = explore.decode_keys(OW1, tag_names)
OW2 = explore.decode_keys(OW2, tag_names)
OW3 = explore.decode_keys(OW3, tag_names)

In [7]:
ts_dfs = [OW1["DHP"], OW1["WHP"], OW1["DHT"], OW1["WHT"], OW1["GLP"]]
ts_labels = ["DHP", "WHP", "DHT", "WHT", "GLP"]

ge2016 = [df.where(df.datetime >= '2016-01-01') for df in ts_dfs]

explore.plot_ts("WHP, WHT, DHP, DHT, GLP over time", "datetime", "value", ge2016, ts_labels)

In [8]:
ts_dfs = [OW1["DHP"], OW1["WHP"], OW1["GLP"]]
ts_labels = ["DHP", "WHP", "GLP"]

ge2016p = [df.where(df.datetime >= '2016-01-01') for df in ts_dfs]

explore.plot_ts("WHP, DHP, GLP over time", "datetime", "value", ge2016p, ts_labels)

In [9]:
ts_dfs = [OW1["DHT"], OW1["WHT"]]
ts_labels = ["DHT", "WHT"]

ge2016t = [df.where(df.datetime >= '2016-01-01') for df in ts_dfs]

explore.plot_ts("WHT, DHT over time", "datetime", "value", ge2016t, ts_labels)

In [10]:
# Load new data containing rates into the notebook
OW1_ql_df = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/Qliq.csv')

# Add the relevant columns to the oilwell dictionary
OW1["LR"] = OW1_ql_df.select(
                                   F.to_timestamp(F.col("DATE").cast("string"), "MM/dd/yyyy").alias("datetime"),
                                    OW1_ql_df["Daily liquid rate [Sm3/d]"].alias("value")
)

OW1["OR"] = OW1_ql_df.select(
                                   F.to_timestamp(F.col("DATE").cast("string"), "MM/dd/yyyy").alias("datetime"),
                                    OW1_ql_df["Daily oil [Sm3/d]"].alias("value")
)

# - Compare if WHP, DHP, GLR are the same as the dump ones!

In [11]:
# Try to find the interventions thats occurred over the years
ts_dfs = [OW1["DHP"], OW1["WHP"], OW1["GLP"], OW1["OR"], OW1["LR"]]
ts_labels = ["DHP", "WHP", "GLP", "OR", "LR"]

ge2016norm = [df.where(df.datetime >= '2016-01-11') for df in ts_dfs]

for i, df in enumerate(ge2016norm):
  mean, std = df.select(F.mean("value"), F.stddev("value")).first()
  ge2016norm[i] = df.withColumn("value_norm", (col("value") - mean) / std)
  ge2016norm[i] = ge2016norm[i].select(ge2016norm[i]["datetime"], ge2016norm[i]["value_norm"].alias("value"))

explore.plot_ts("Normalised WHP, DHP, GLP, OR, LR over time", "datetime", "value", ge2016norm, ts_labels)

In [12]:
# Try to find the interventions thats occurred over the years
ts_dfs = [OW1["DHP"], OW1["WHP"], OW1["LR"]]
ts_labels = ["DHP", "WHP", "LR"]

ge2016norm = [df.where(df.datetime >= '2016-01-11') for df in ts_dfs]

for i, df in enumerate(ge2016norm):
  mean, std = df.select(F.mean("value"), F.stddev("value")).first()
  ge2016norm[i] = df.withColumn("value_norm", (col("value") - mean) / std)
  ge2016norm[i] = ge2016norm[i].select(ge2016norm[i]["datetime"], ge2016norm[i]["value_norm"].alias("value"))

explore.plot_ts("Normalised WHP, DHP, LR over time", "datetime", "value", ge2016norm, ts_labels)

In [13]:
# Try to find the interventions thats occurred over the years
ts_dfs = [OW1["DHP"], OW1["WHP"], OW1["LR"]]
ts_labels = ["DHP", "WHP", "LR"]

ge2016norm = [df.where(df.datetime >= '2016-01-11') for df in ts_dfs]

for i, df in enumerate(ge2016norm):
  mean, std = df.select(F.mean("value"), F.stddev("value")).first()
  ge2016norm[i] = df.withColumn("value_norm", (col("value")) / std)
  ge2016norm[i] = ge2016norm[i].select(ge2016norm[i]["datetime"], ge2016norm[i]["value_norm"].alias("value"))

explore.plot_ts("Standardised WHP, DHP, LR over time", "datetime", "value", ge2016norm, ts_labels)

In [14]:
# Import in the data:
df_records = spark.read.format('csv').options(header='true', inferSchema='true', delimiter='|', encoding='iso-8859-1').load('/FileStore/tables/interences_filtered.csv')

# Translate column to english!
df_records, trans_dict = explore.translate_col("Description", df_records)

# Pick out only date and description columns
OW1["Records"] = df_records.select(
                 F.to_timestamp(F.col("Date").cast("string"), "MM/dd/yyyy").alias("datetime"),
                 df_records["Description"]
)

# recorded interfaces begin 2017-01-20
OW1["WHP_2017_2019"] = OW1["WHP"].where(OW1["WHP"].datetime >= '2017-01-20')

abrv_dict = {
            "due to sand" : "Choking due to sand",
            "Valve test" : "Valve test"
}

# Overlay time series graph with the records
OW1["WHP_2017_2019"] = explore.ts_overlay_records(OW1["WHP_2017_2019"], OW1["Records"], "Description", filt_dict=abrv_dict, translate_dict=trans_dict)

# Remove any duplicates, and merge any related comments
OW1["WHP_2017_2019"] = explore.merge_duplicate(OW1["WHP_2017_2019"])

# Produce a discretisation of the columns
OW1["WHP_2017_2019"] = explore.discretise_col(OW1["WHP_2017_2019"], "Grouped")

display(OW1["WHP_2017_2019"])

In [15]:
# Add a year column
OW1["WHP_2017_2019"] = explore.add_year_col(OW1["WHP_2017_2019"])

# Seperate dataframe by year
years = ['2017', '2018', '2019']
dfs_years = [OW1["WHP_2017_2019"].where(OW1["WHP_2017_2019"].year == y).toPandas() for y in years]

fig, axs = plt.subplots(3, 1, figsize=(24, 24))
axs.flatten()

for df, ax, year in zip(dfs_years, axs, years):
  groups = df.groupby('Grouped')
  ax.plot(df.datetime, df.value, marker='', linestyle='-')
  for name, group in groups:
    if name == "No Records": continue
    ax.plot(group.datetime, group.value, marker='o', linestyle='', label=name)
  ax.legend(loc="best")
  ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d %H:%M"))
  ax.xaxis.set_minor_formatter(mdates.DateFormatter("%Y-%m-%d"))
  ax.set_title(year, fontsize=16)
  ax.set_xlabel("datetime", fontsize=16)
  ax.set_ylabel("value", fontsize=16)
  ax.grid(True)

fig.tight_layout(pad=0.4, w_pad=0.5, h_pad=3.0)
display(fig)

In [16]:
display(OW1["WHP_2017_2019"])

In [17]:
offsets =  [i for i in range(24)] 
weights = [24 for i in range(24)]
print(len(offsets))
print(len(weights))

In [18]:
OW1["WHP_2017_2019"] = OW1["WHP_2017_2019"].withColumn("day", F.date_trunc("day", OW1["WHP_2017_2019"].datetime))

df_OW1 = OW1["WHP_2017_2019"]\
              .groupBy("day")\
              .agg(F.avg("value"))\
              .orderBy("day")

display(df_OW1)

In [19]:
df_OW1 = df_OW1.select(df_OW1.day.alias("datetime"), df_OW1["avg(value)"].alias("value"))

ts_dfs = [df_OW1]
ts_labels = ["WHP"]

explore.plot_ts("WHP daily averaged over time", "datetime", "value", ts_dfs, ts_labels)
# view_moving_avg("Moving average of OW1 WHP", "WHP (Bar)", OW1["GLR"], offsets, weights)

In [21]:
years = ['2017', '2018', '2019']
dfs_years = [new_df1.where(new_df1.year == y).toPandas() for y in years]

fig, axs = plt.subplots(3,1, figsize=(24, 24))
axs.flatten()

for df, ax in zip(dfs_years, axs):
  groups = df.groupby('grouped_desc')
  ax.plot(df.datetime, df.value, marker='', linestyle='-')
  for name, group in groups:
    ax.plot(group.datetime, group.value, marker='o', linestyle='', label=name)
    ax.grid(True)
  ax.legend(loc="best")
  ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d %H:%M"))
  ax.xaxis.set_minor_formatter(mdates.DateFormatter("%Y-%m-%d"))
  ax.set_title("title", fontsize=16)
  ax.set_xlabel("x_head", fontsize=16)
  ax.set_ylabel("y_head", fontsize=16)

fig.tight_layout()
display(fig)